## Home assignment 05: Bagging and OOB score

Please, fill the lines in the code below.
This is a simplified version of `BaggingRegressor` from `sklearn`. Please, notice, that `sklearn` API is **not preserved**.

Your algorithm should be able to train different instances of the same model class on bootstrapped datasets and to provide [OOB score](https://en.wikipedia.org/wiki/Out-of-bag_error) for the training set.

The model should be passed as model class with no explicit parameters and no parentheses.

Example:
```
import numpy as np
from sklearn.linear_model import LinearRegression

bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
bagging_regressor.fit(LinearRegression, X, y)

```

In [1]:
import numpy as np

In [39]:
"""
import numpy as np

class SimplifiedBaggingRegressor:
    def __init__(self, num_bags, oob=False):
        self.num_bags = num_bags
        self.oob = oob
        
    def _generate_splits(self, data: np.ndarray):
        '''
        Generate indices for every bag and store in self.indices_list list
        '''
        self.indices_list = []
        data_length = len(data)
        for bag in range(self.num_bags):
            self.indices_list.append(np.random.randint(low=0, 
                                                       high=data_length,
                                                       size=data_length))


    def fit(self, model_constructor, data, target):
        '''
        Fit model on every bag.
        Model constructor with no parameters (and with no ()) is passed to this function.
        
        example:
        
        bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
        bagging_regressor.fit(LinearRegression, X, y)
        '''
        self.data = None
        self.target = None
        self._generate_splits(data)
        assert len(set(list(map(len, self.indices_list)))) == 1, 'All bags should be of the same length!'
        assert list(map(len, self.indices_list))[0] == len(data), 'All bags should contain `len(data)` number of elements!'
        self.models_list = []
        for bag in range(self.num_bags):
            model = model_constructor()
            data_bag, target_bag = data[self.indices_list[bag]], target[self.indices_list[bag]] # Your Code Here
            self.models_list.append(model.fit(data_bag, target_bag)) # store fitted models here
        if self.oob:
            self.data = data
            self.target = target
        
    def predict(self, data):
        '''
        Get average prediction for every object from passed dataset
        '''
        '''
        predictions = np.zeros(len(data))
        for model in self.models_list:
            predictions += model.predict(data)
            #preds.append(model.predict(data))
        '''
        return [model.predict(data) for model in self.models_list]
        

    
    def _get_oob_predictions_from_every_model(self):
        '''
        Generates list of lists, where list i contains predictions for self.data[i] object
        from all models, which have not seen this object during training phase
        '''
        list_of_predictions_lists = [[] for _ in range(len(self.data))]
        list_of_predictions_lists = [[model.predict(self.data[i].reshape(1, -1)).item() for model in np.array(self.models_list)[np.array([i not in val for val in self.indices_list])]] for i in range(len(self.data))]
        '''
        for i in range(len(self.data)):
            sample = self.data[i].reshape(1,-1)
            predictions = []
            for bag in range(self.num_bags):
                if i not in self.indices_list[bag]:
                    predictions.append(self.models_list[bag].predict(sample))
            list_of_predictions_lists[i] = predictions
        '''
        self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)
        
    
    def _get_averaged_oob_predictions(self):
        '''
        Compute average prediction for every object from training set.
        If object has been used in all bags on training phase, return None instead of prediction
        '''
        self._get_oob_predictions_from_every_model()

        self.oob_predictions = [np.mean(el) if len(el) > 0 else None for el in self.list_of_predictions_lists]
        
        
    def OOB_score(self):
        '''
        Compute mean square error for all objects, which have at least one prediction
        '''
        self._get_averaged_oob_predictions()

        return np.mean([(x[0] - x[1])**2 for x in zip(self.oob_predictions, self.target) if x[0] is not None])

"""
import numpy as np

class SimplifiedBaggingRegressor:
    def __init__(self, num_bags, oob=False):
        self.num_bags = num_bags
        self.oob = oob
        
    def _generate_splits(self, data: np.ndarray):
        '''Generate indices for every bag and store in self.indices_list list'''
        
        self.indices_list = []
        data_length = len(data)
        
        for bag in range(self.num_bags):
            self.indices_list.append(np.random.randint(low=0, 
                                                       high=data_length,
                                                       size=data_length))


    def fit(self, model_constructor, data, target):
        '''
        Fit model on every bag.
        Model constructor with no parameters (and with no ()) is passed to this function.
        
        example:
        
        bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
        bagging_regressor.fit(LinearRegression, X, y)
        '''
        
        self.data = None
        self.target = None
        self._generate_splits(data)
        
        assert len(set(list(map(len, self.indices_list)))) == 1, 'All bags should be of the same length!'
        assert list(map(len, self.indices_list))[0] == len(data), 'All bags should contain `len(data)` number of elements!'
        
        self.models_list = []
        
        for bag in range(self.num_bags):
            data_bag, target_bag = data[self.indices_list[bag]], target[self.indices_list[bag]] # Your Code Here
            self.models_list.append(model_constructor().fit(data_bag, target_bag)) # store fitted models here
        if self.oob is True:
            self.data = data
            self.target = target
        
    def predict(self, data):
        '''
        Get average prediction for every object from passed dataset
        '''
        result_predictions = []
        for model in self.models_list:
            result_predictions.append(model.predict(data))
        return result_predictions

        
    
    def _get_oob_predictions_from_every_model(self):
        '''
        Generates list of lists, where list i contains predictions for self.data[i] object
        from all models, which have not seen this object during training phase
        '''
        list_of_predictions_lists = [[] for _ in range(len(self.data))]


        
        for i in range(len(self.data)):
            
            predictions = []
            
            for bag in range(self.num_bags):
            
                if i not in self.indices_list[bag]:
            
                    predictions.append(float(self.models_list[bag].predict(self.data[i].reshape(1, -1))))
            
            list_of_predictions_lists[i] = predictions
        
        self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)
        
    
    def _get_averaged_oob_predictions(self):
        '''
        Compute average prediction for every object from training set.
        If object has been used in all bags on training phase, return None instead of prediction
        '''
        self._get_oob_predictions_from_every_model()
        self.oob_predictions = []
        
        for elem in self.list_of_predictions_lists:
        
            if len(elem) > 0:
                self.oob_predictions.append(np.mean(elem))
        
            else:
                self.oob_predictions.append(None)
        
        
    def OOB_score(self):
        '''
        Compute mean square error for all objects, which have at least one prediction
        '''
        self._get_averaged_oob_predictions()
        result = []
        for elem in range(len(self.oob_predictions)):
            if self.oob_predictions[elem] is not None:
                result.append((self.oob_predictions[elem] - self.target[elem])**2)
        return np.mean(result)


### Local tests:

In [3]:
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm

/Users/aleksandrkozuhov/anaconda3/envs/neural_nets/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Simple tests:

In [41]:
import warnings
warnings.filterwarnings("ignore")

for _ in tqdm(range(100)):
    X = np.random.randn(2000, 10)
    y = np.mean(X, axis=1)
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    assert np.mean((predictions - y)**2) < 1e-6, 'Linear dependency should be fitted with almost zero error!'
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score < 1e-6, 'OOB error for linear dependency should be also close to zero!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Simple tests done!')

  1%|          | 1/100 [00:00<00:42,  2.36it/s]

1.457712463115084e-31


  2%|▏         | 2/100 [00:00<00:43,  2.23it/s]

1.6431237158961804e-31


  3%|▎         | 3/100 [00:01<00:49,  1.95it/s]

...

1.4506631247882511e-31


 99%|█████████▉| 99/100 [00:37<00:00,  2.84it/s]

1.1628470004146424e-31


100%|██████████| 100/100 [00:37<00:00,  2.65it/s]

8.65184616151438e-32
Simple tests done!


#### Medium tests

In [46]:
for _ in tqdm(range(10)):
    X = np.random.randn(200, 150)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=20, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    average_train_error = np.mean((predictions - y)**2)
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score > average_train_error, 'OOB error must be higher than train error due to overfitting!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Medium tests done!')

100%|██████████| 10/10 [00:02<00:00,  4.66it/s]

Medium tests done!


#### Complex tests:

In [43]:
for _ in tqdm(range(10)):
    X = np.random.randn(2000, 15)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=100, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    oob_score = bagging_regressor.OOB_score()
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 1e-2, 'Probability of missing a bag should be close to theoretical value!'
    
print('Complex tests done!')

100%|██████████| 10/10 [00:21<00:00,  2.18s/it]

Complex tests done!


In [34]:
np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)

-0.00013444117144228995

Great job! Please, save `SimplifiedBaggingRegressor` to  `bagging.py` and submit your solution to the grading system!